In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [41]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from keras.preprocessing import text as keras_text, sequence as keras_seq
from keras.utils import np_utils
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.layers import Dense, Activation, Convolution1D, MaxPooling1D, Dropout, Flatten
from keras.models import Sequential, save_model
from sklearn.model_selection import train_test_split
from keras.layers import Embedding
from sklearn.metrics import classification_report,confusion_matrix
import tensorflow as tf

In [17]:
#nltk.download('stopwords')

In [18]:
#Tokenizer=RegexpTokenizer(r"\w+[a-z]+")   ## only word
#word=set(stopwords.words("english"))

In [19]:
data_new=pd.read_csv('english_dataset.tsv', sep = '\t', encoding="latin-1")

In [20]:
data_new.shape

(5852, 5)

In [21]:
data_new=data_new.dropna()
data_new.shape

(5852, 5)

In [22]:
data_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5852 entries, 0 to 5851
Data columns (total 5 columns):
text_id    5852 non-null object
text       5852 non-null object
task_1     5852 non-null object
task_2     5852 non-null object
task_3     5852 non-null object
dtypes: object(5)
memory usage: 274.3+ KB


In [23]:
data_new.text=data_new.text.astype(str)

In [24]:
df2=data_new['text']
labels=data_new['task_1']

In [25]:
data_new=data_new[['text','task_1']]

In [26]:
from keras.preprocessing.text import one_hot

In [27]:
#df_text = data_new['text']
data_new

,text,task_1
0,#DhoniKeepsTheGlove | WATCH: Sports Minister K...,NOT
1,@politico No. We should remember very clearly ...,HOF
2,@cricketworldcup Guess who would be the winner...,NOT
3,Corbyn is too politically intellectual for #Bo...,NOT
4,All the best to #TeamIndia for another swimmin...,NOT
5,@kellymiller513 @TheRealOJ32 I hope you rememb...,NOT
6,@ICC Latest design of #WC2019 trophy. #CWC2019...,NOT
7,#ADOS #trendingnow #blacklivesmatter #justice ...,HOF
8,Thanks for your support! Wow 600k. Graffiti ha...,NOT
9,By wearing the #BalidaanBadge over his gloves ...,NOT


In [28]:
data_new['task_1'] = data_new['task_1'].map({'NOT': 0, 'HOF': 1})
data_new

,text,task_1
0,#DhoniKeepsTheGlove | WATCH: Sports Minister K...,0
1,@politico No. We should remember very clearly ...,1
2,@cricketworldcup Guess who would be the winner...,0
3,Corbyn is too politically intellectual for #Bo...,0
4,All the best to #TeamIndia for another swimmin...,0
5,@kellymiller513 @TheRealOJ32 I hope you rememb...,0
6,@ICC Latest design of #WC2019 trophy. #CWC2019...,0
7,#ADOS #trendingnow #blacklivesmatter #justice ...,1
8,Thanks for your support! Wow 600k. Graffiti ha...,0
9,By wearing the #BalidaanBadge over his gloves ...,0


In [33]:
vocab_size = 4000000
# encode full sentence into vector
encoded_docs=[one_hot(d,vocab_size) for d in df2]
print (encoded_docs)

[[2848342, 3761103, 2979764, 1465719, 2347000, 619380, 1656451, 72133, 5343, 3374764, 827149, 3010782, 45981, 3674600, 1432501, 3183889, 2425979, 191899, 3325995, 3360945, 2464821, 790314, 3301661, 3521510, 955812, 164745, 1467109, 3360945, 2379950, 3860254, 3688053, 3492931, 2458531, 2299065, 3640626, 822729, 3825501, 2758396], [172146, 3105177, 545051, 846689, 545969, 3175950, 746387, 2478712, 2566010, 3170735, 1786785, 2425979, 1262129, 906486, 942761, 1668358], [160211, 299778, 2804366, 913938, 2743351, 3360945, 1697886, 899660, 199442, 815881, 451700, 2804366, 1374794, 1051649, 1227674, 1419600, 3360945, 2678812, 1216656, 2582639, 789810, 515155, 3301661], [2500855, 2458531, 772655, 282180, 3221401, 1280485, 2242175, 2716056, 2508597, 1468108, 3640626, 822729, 3825501, 2788573], [872375, 3360945, 405547, 2425979, 788901, 1280485, 1981470, 950326, 3809299, 785349, 2814451, 3109219, 1668831, 2826579, 789810, 815881, 3412387, 3851566, 3640626, 822729, 3825501, 1012374], [1258167, 446

In [42]:
# pad documents to a max length of 100 words
import keras
max_length = 100
padded_docs = tf.keras.preprocessing.sequence.pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[2848342 3761103 2979764 ...       0       0       0]
 [ 172146 3105177  545051 ...       0       0       0]
 [ 160211  299778 2804366 ...       0       0       0]
 ...
 [ 644975  926633 2215569 ...       0       0       0]
 [1943652 3157275 2376865 ...       0       0       0]
 [ 788623  199442 2743351 ...       0       0       0]]


In [43]:
from keras.utils import to_categorical
labels=data_new['task_1']
labels= to_categorical(labels,2)
labels

array([[1., 0.],
       [0., 1.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [45]:
x_train, x_test, y_train, y_test = train_test_split(padded_docs,labels, test_size=0.20, random_state=42)

In [46]:
model = Sequential()
model.add(Embedding(vocab_size, 8, input_length=max_length))
model.add( Convolution1D(8, 4 ))
model.add( Activation('relu') )

model.add( Convolution1D(16,4) )

model.add( Activation('relu'))
model.add(Convolution1D(64,4))
model.add( MaxPooling1D( pool_size=(4) ) )
model.add(Activation("relu"))
model.add(Convolution1D(64,4))
model.add(Activation("relu"))

model.add( Flatten() )

model.add( Dense(2) )

model.add( Activation('sigmoid') )

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 8)            32000000  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 97, 8)             264       
_________________________________________________________________
activation_1 (Activation)    (None, 97, 8)             0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 94, 16)            528       
_________________________________________________________________
activation_2 (Activation)    (None, 94, 16)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 91, 64)            4160      
_________________________________________________________________
max_

In [82]:
from keras.callbacks import ModelCheckpoint
filepath = "task1_cnn_onehot_100d.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor="val_acc", verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [83]:
model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['acc'])

In [84]:
#hist=model.fit(x_train,y_train,epochs=100,batch_size=32,validation_data=(x_test, y_test))
hist=model.fit(x_train,y_train,epochs=100,batch_size=32,validation_data=(x_test, y_test),callbacks=callbacks_list)

Train on 4681 samples, validate on 1171 samples
Epoch 1/100
4681/4681 [==============================] - 85s 18ms/step - loss: 0.6703 - acc: 0.6101 - val_loss: 0.6533 - val_acc: 0.6268

Epoch 00001: val_acc improved from -inf to 0.62681, saving model to task1_cnn_onehot_100d.hdf5
Epoch 2/100
4681/4681 [==============================] - 86s 18ms/step - loss: 0.6192 - acc: 0.6567 - val_loss: 0.6780 - val_acc: 0.5760

Epoch 00002: val_acc did not improve from 0.62681
Epoch 3/100
4681/4681 [==============================] - 86s 18ms/step - loss: 0.4934 - acc: 0.7702 - val_loss: 0.6936 - val_acc: 0.5820

Epoch 00003: val_acc did not improve from 0.62681
Epoch 4/100
4681/4681 [==============================] - 88s 19ms/step - loss: 0.3720 - acc: 0.8424 - val_loss: 0.8177 - val_acc: 0.6652

Epoch 00004: val_acc improved from 0.62681 to 0.66524, saving model to task1_cnn_onehot_100d.hdf5
Epoch 5/100
4681/4681 [==============================] - 85s 18ms/step - loss: 0.2778 - acc: 0.8867 - val_l


Epoch 00043: val_acc did not improve from 0.66524
Epoch 44/100
4681/4681 [==============================] - 89s 19ms/step - loss: 0.0096 - acc: 0.9971 - val_loss: 2.5244 - val_acc: 0.5978

Epoch 00044: val_acc did not improve from 0.66524
Epoch 45/100
4681/4681 [==============================] - 90s 19ms/step - loss: 0.0047 - acc: 0.9967 - val_loss: 3.5355 - val_acc: 0.5730

Epoch 00045: val_acc did not improve from 0.66524
Epoch 46/100
4681/4681 [==============================] - 91s 19ms/step - loss: 0.0055 - acc: 0.9967 - val_loss: 2.1868 - val_acc: 0.6016

Epoch 00046: val_acc did not improve from 0.66524
Epoch 47/100
4681/4681 [==============================] - 90s 19ms/step - loss: 0.0056 - acc: 0.9971 - val_loss: 2.8843 - val_acc: 0.5952

Epoch 00047: val_acc did not improve from 0.66524
Epoch 48/100
4681/4681 [==============================] - 88s 19ms/step - loss: 0.0058 - acc: 0.9972 - val_loss: 2.7826 - val_acc: 0.6213

Epoch 00048: val_acc did not improve from 0.66524
Epoc

In [85]:
from keras.models import load_model
# load weights into new model
loaded_model= load_model("task1_cnn_onehot_100d.hdf5")

In [86]:
w=loaded_model.predict_classes(x_test)

In [87]:
from sklearn.metrics import classification_report,confusion_matrix

In [88]:
Y_actual=[]
for ix in y_test:
    Y_actual.append(np.argmax(ix))

In [89]:
print(classification_report(Y_actual,w))

             precision    recall  f1-score   support

          0       0.69      0.86      0.76       734
          1       0.59      0.33      0.43       437

avg / total       0.65      0.67      0.64      1171



In [90]:
print(confusion_matrix(Y_actual, w))

[[634 100]
 [291 146]]


In [91]:
data_test=pd.read_csv('hasoc2019_en_test.tsv', sep = '\t', encoding="latin-1")

In [17]:
data_test.head()

651

In [92]:
data_test.text=data_test.text.astype(str)
df_text=data_test['text']

In [93]:
vocab_size = 4000000
# encode full sentence into vector
encoded_docs_test=[one_hot(d,vocab_size) for d in df_text]
print (encoded_docs_test)

[[633454, 353559, 1238048, 3523494, 543436, 1346336, 3662602, 2994218, 2857539, 2184973, 93633, 3777555, 116733, 2312458, 3612429, 2808844, 3008462, 263825, 2894803, 2154859, 314467, 751591], [2199975, 3861710, 903744, 3781148, 2038339, 2912103, 2525074, 2994218, 297107, 1521081, 869815, 420171, 2229611, 3417938, 2978407, 2420432, 3231217, 2547852, 288142], [1515470, 3064502, 1515470, 633323, 2706926, 2108190, 2662090, 8967, 481135, 1297266, 2388545, 1200105], [3276025, 538925, 3195656, 2343568, 524316, 2936666, 177340, 2852919, 3432905, 481135, 1922363, 866147, 2108190, 1103497, 1003228, 877567, 3809945, 1025338, 3355334, 1183361, 504494], [858725, 1515470, 1009376, 3043060, 2994218, 481135, 2383159, 2631273, 1515470, 205232, 597878, 1198163, 1971180, 916392, 2263847, 1647779, 3004368, 3341856, 3791331, 2184973], [331603, 143934, 1778763, 59175, 2891247, 271105, 2994218, 1369643, 3046930, 1951143, 2569820, 2899875, 2604479, 1647779, 3010562, 3777555, 3195656, 3550359, 3417938, 1723666

In [94]:
# pad documents to a max length of 4 words
max_length = 100
padded_docs_test = pad_sequences(encoded_docs_test, maxlen=max_length, padding='post')
print(padded_docs_test)

[[ 633454  353559 1238048 ...       0       0       0]
 [2199975 3861710  903744 ...       0       0       0]
 [1515470 3064502 1515470 ...       0       0       0]
 ...
 [2327666 2312458 1555237 ...       0       0       0]
 [ 481135 2321285  132827 ...       0       0       0]
 [2839522 2448596 2994218 ...       0       0       0]]


In [95]:
w_test=loaded_model.predict_classes(padded_docs_test)

In [22]:
w_test

In [96]:
print(len(w_test)-np.count_nonzero(w_test))
np.count_nonzero(w_test)

815


338

In [97]:
output = pd.DataFrame()
#out_result['text_id'] = 

In [99]:
output['text_id'] = data_test['text_id']
output.tail()

,text_id
1148,hasoc_en1_3958
1149,hasoc_en1_4648
1150,hasoc_en1_4832
1151,hasoc_en1_3721
1152,hasoc_en1_991


In [100]:
ans_dict = {0:'NOT',1:'HOF'}
w2 = np.vectorize(ans_dict.get)(w_test)
output['result'] = w2

In [101]:
output.to_csv('eng_task1_CNN_onehot_100d.tsv',header=True,sep='\t',index=False)